In [2]:
# load package
# string 
import re

# math
import pandas as pd
import numpy as np
import scipy as sp
import random

# sys
import sys
import os
import time
import warnings

# machine learning
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, gaussian_process, discriminant_analysis
from xgboost import XGBClassifier

# model utils
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection 
from sklearn import model_selection
from sklearn import metrics

# plot
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix #??
# = show plots in Jupyter Notebook browser
%matplotlib inline 
mpl.style.use('ggplot') #??
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8 #??

# self define
sys.path.append('../../utils/')
import Prepare
import Overview
import Completing
import Correcting
import Modelling

In [8]:
Overview.show_tools()


    following is same methods that often used in overlook step
    d_train = pd.read_csv('dir')
    d_train.info() # show number of total entries, features type and number
                   # we should code the object features and fill the null value
    d_train.sample(10)
    d_train.head(10)
    d_train.tail(10)
    d_train.describe() # list basic statistic value for numeric feature
    d_train.describe(include = 'all')

    besides we can also plot the feature the get a better viewing, for a viry first look, we always instrested in the distibution and target rate
    import matplotlib.pyplot as plt
    import seaborn as sns
    f, ax = plt.subplot(3,4,figsize = (20,16))
    # count the number with countplot
    sns.countplot('Pclass', data = d_train, ax = ax[0,0])
    # view the target rate by adding a third dimension hue
    sns.countplot('SibSp', hue = 'Survived', data=train, ax = ax[0,3], palette = 'hus1')
    sns.boxplot(x='Pclass', y= 'Age', data = train, ax = ax[0,2])
    # 

### Overview

In [3]:
item_cat = pd.read_csv('../data/item_categories.csv')
items = pd.read_csv('../data/items.csv')
shops = pd.read_csv('../data/shops.csv')
sales_train = pd.read_csv('../data/sales_train.csv')
sales_test = pd.read_csv('../data/test.csv.gz')

In [10]:
item_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 2 columns):
item_category_name    84 non-null object
item_category_id      84 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.4+ KB


In [11]:
item_cat

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4
5,Аксессуары - PSVita,5
6,Аксессуары - XBOX 360,6
7,Аксессуары - XBOX ONE,7
8,Билеты (Цифра),8
9,Доставка товара,9


In [12]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22170 entries, 0 to 22169
Data columns (total 3 columns):
item_name           22170 non-null object
item_id             22170 non-null int64
item_category_id    22170 non-null int64
dtypes: int64(2), object(1)
memory usage: 519.7+ KB


In [13]:
items.sample(15)

,item_name,item_id,item_category_id
14952,Маленькие гелевые наклейки хеллоуин,14952,66
7941,Xbox 360 250 ГБ Kinect bundle Fable The Journe...,7941,15
430,1С:Аудиотеатр. Коллекция сказок А. Усачева (Di...,430,45
20306,ФОРСАЖ. ПЕНТАЛОГИЯ,20306,41
15659,НЕИЗВЕСТНЫЙ,15659,40
15981,Наклейки одевайка прогулка 2 арт.WMS0013,15981,66
12446,КЕВИН СПЕЙСИ (ТОМ 1) (регион),12446,40
11217,ДРУГАЯ ЖЕНЩИНА (BD),11217,37
6354,STING A Winter's Night - Live From Durham Cath...,6354,59
8181,"АКВАРИУМ Пушкинская, 10 LP",8181,58


In [14]:
shops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
shop_name    60 non-null object
shop_id      60 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.0+ KB


In [15]:
shops.sample(15)

,shop_name,shop_id
34,"Н.Новгород ТРЦ ""РИО""",34
9,Выездная Торговля,9
25,"Москва ТРК ""Атриум""",25
20,"Москва ""Распродажа""",20
32,"Москва ТЦ ""Серебряный Дом""",32
40,"РостовНаДону ТРК ""Мегацентр Горизонт"" Островной",40
38,"Омск ТЦ ""Мега""",38
55,Цифровой склад 1С-Онлайн,55
30,"Москва ТЦ ""Перловский""",30
33,"Мытищи ТРК ""XL-3""",33


In [16]:
sales_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
date              object
date_block_num    int64
shop_id           int64
item_id           int64
item_price        float64
item_cnt_day      float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


In [17]:
sales_train.sample(15)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
264340,29.03.2013,2,54,22136,149.0,1.0
1149111,14.12.2013,11,30,6075,1499.0,1.0
1548297,12.03.2014,14,7,2946,599.0,1.0
1940254,03.08.2014,19,30,14831,699.0,2.0
2530591,21.03.2015,26,2,17046,399.0,1.0
2143300,27.11.2014,22,15,12974,249.0,1.0
1124299,14.11.2013,10,50,3656,299.0,1.0
2824799,12.08.2015,31,24,11785,799.0,1.0
876821,07.09.2013,8,4,5592,2990.0,1.0
2504833,03.03.2015,26,56,8808,99.0,1.0


In [18]:
sales_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 3 columns):
ID         214200 non-null int64
shop_id    214200 non-null int64
item_id    214200 non-null int64
dtypes: int64(3)
memory usage: 4.9 MB


In [19]:
sales_test.sample(15)

,ID,shop_id,item_id
175324,175324,35,8035
140329,140329,57,18375
201492,201492,44,2531
150590,150590,59,18759
138220,138220,57,1486
32305,32305,10,2884
74475,74475,21,1290
68224,68224,24,8035
150752,150752,59,4653
78754,78754,15,20536


In [5]:
sales_train[sales_train['item_id'] == 11365].sort_values(['item_price'])

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
1651714,16.05.2014,16,12,11365,124.000000,5.0
2805487,21.08.2015,31,12,11365,170.000000,2.0
1330776,13.01.2014,12,12,11365,180.000000,3.0
1398688,25.02.2014,13,12,11365,194.000000,5.0
661581,05.07.2013,6,12,11365,230.000000,4.0
2731490,30.07.2015,30,12,11365,230.000000,1.0
1398683,06.02.2014,13,12,11365,246.666667,9.0
2626123,21.05.2015,28,12,11365,248.333333,6.0
2608316,16.04.2015,27,12,11365,249.333333,15.0
1058265,29.11.2013,10,12,11365,260.867925,53.0


In [6]:
sales_train['item_cnt_day'][2909818]

2169.0

In [7]:
test_nrow = sales_test.shape[0]

In [9]:
sales_test.shape

(214200, 3)

In [10]:
sales_train['item_cnt_day'].describe()

count    2.935849e+06
mean     1.242641e+00
std      2.868475e+03
min     -2.200000e+01
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      2.169000e+03
Name: item_cnt_day, dtype: float64